## README:
This file contains a class called `cleanData` that is responsible for running the basic cleanings for the course datasets.\
Inside cleanData, functions can perform the following tasks:
- asserts whether current file is a csv
- reads the csv file
- extracts the list of quantitative questions from the question dictionary
- transforms numerical values that are identified as strings into numbers
- joins necessary columns needed for the faculty dashboard together, columns include:
    - `Email`
    - `TeamNumber`
    - `TeammateNumber`
    - `RecipientLastName`
    - `RecipientFirstName`
    - `RecipientEmail` (repeating `Email` column; will remove later)
- maps the agree/disagree scale to number scale
    - survey answer format will change (?) so consider removing this function later
- normalize the data on scale for visualization purpose
- sorts the numerical columns in the dataset\
$\vdots$\
more coming soon
  
For each individual courses, its dataset will first be passed into `cleanData.main()` for basic cleaning.\
If specific aspects/questions requested, then create a separate class to address in detail to maintain abstraction.

E.g. for ME292C, we are asked to rank team performance:
- performance across all questions
- performance split by question category

In [133]:
import pandas as pd
import numpy as np

In [134]:
# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

In [161]:
class cleanData:
    def __init__(self, raw_data, question_dictionary, roster):
        self.raw_data = raw_data
        self.question_dict = question_dictionary
        self.roster = roster
        
        self.subset_data = raw_data
        self.needsNormalization = True
        self.needsMapping = True
    
    # assert all files are of .csv extension 
    def assertCSVFiles(self):
        if not "CSV" in self.raw_data.upper():
            raise Exception("raw data file is not in CSV format")
        if not "CSV" in self.roster.upper():
            raise Exception("roster file is not in CSV format")
        if not "CSV" in self.question_dict.upper():
            raise Exception("question dictionary file is not in CSV format")
        
        # if still want to check all files together
        # if not any(["CSV" in [self.raw_data.upper(), self.roster.upper(), self.question_dict.upper()]]):
        #     raise Exception("At least one of the input files is not in CSV format")

    # read all CSV files 
    def readCSVs(self):
        self.raw_data = pd.read_csv(self.raw_data)
        self.question_dict = pd.read_csv(self.question_dict)
        self.roster = pd.read_csv(self.roster)

    # return the list of quantitative questions from question_dictionary
    def getQuantQuestions(self):
        quant_dict = self.question_dict[self.question_dict["type"] == "quantitative"]
        return list(quant_dict["question_id"])

    # subset raw data to just student email, student first name, student last name, and all question responses 
    def getSubset(self):
        cols_needed = []
        keywords_list = ['EMAIL', 'FIRST', 'LAST', 'Q']
        for col in list(self.raw_data.columns):
            if any(keyword in col.upper() for keyword in keywords_list):
                cols_needed.append(col)

        # update class variable subset_data to the correct subset
        self.subset_data = self.raw_data[cols_needed]

        # replace question column names in subset data with X.Y instead of QX_Y
        self.subset_data.columns = [col.replace('Q', '').replace('_', '.') for col in list(self.subset_data.columns)]
        return self.subset_data

    # join in TeamNumber and TeammateNumber from roster; drop rows of metadata
    def joinRosterAndRaw(self):
        roster_email_field = [col for col in list(self.roster.columns) if 'EMAIL' in col.upper()][0]
        cleaned_email_field = [col for col in list(self.getSubset().columns) if 'EMAIL' in col.upper()][0]

        # joins roster (Email) and raw (RecipientEmail)'s subset data on same email address
        full_cleaned = pd.merge(self.roster[[roster_email_field, 'TeamNumber', 'TeammateNumber']], 
                                self.subset_data, 
                                how="outer", 
                                left_on = roster_email_field, 
                                right_on = cleaned_email_field)
        
        # drop N/A rows, and sort df by TeamNumber then TeammateNumber starting from Team1
        full_cleaned = full_cleaned[~full_cleaned["TeamNumber"].isna()]
        full_cleaned = full_cleaned.sort_values(["TeamNumber", "TeammateNumber"]).reset_index().drop("index", axis=1)

        # removes redundancy of the 2 emails columns 
        full_cleaned = full_cleaned.drop('RecipientEmail', axis=1)
        return full_cleaned
        
    # fix numbers in quantitative questions that are identified as strings into integers
    def stringToNumerical(self, data):
        for question in self.getQuantQuestions():
            question_str = str(question)
            lst = data[question_str].str.contains(r'\d+')
            filtered_lst = [x for x in lst if not (x is np.nan)]
            if any(filtered_lst):
                data[question_str] = pd.to_numeric(data[question_str])   
        return data

    def sortColumns(self, data):
        # transform data columns into float for sorting purpose
        for column in data.columns[5:]:  # 6 can change based on joinRosterAndRaw()
            data.rename(columns={column: float(column)}, inplace=True)

        sorted = data.iloc[:, 5:]
        sorted = sorted.sort_index(axis=1)
        sorted['Email'] = data['Email']
        data = data.iloc[:, :5]    
        
        return pd.merge(data, sorted, on='Email')
        
    def mappingScale(self, maxScale):
        seven_scale_mappings = {
        "STRONGLY AGREE" : 7,
        "AGREE" : 6,
        "SOMEWHAT AGREE" : 5,
        "NEITHER AGREE NOR DISAGREE" : 4,
        "SOMEWHAT DISAGREE" : 3,
        "DISAGREE" : 2,
        "STRONGLY DISAGREE" : 1,
        np.nan : np.nan,
        "MUCH BETTER" : 7,
        "MODERATELY BETTER" : 6,
        "SLIGHTLY BETTER" : 5,
        "ABOUT THE SAME" : 4,
        "SLIGHTLY WORSE" : 3,
        "MODERATELY WORSE" : 2,
        "MUCH WORSE" : 1,
        }

        five_scale_mappings = {
            "ALWAYS" : 5,
            "VERY OFTEN" : 4,
            "SOMETIMES" : 3,
            "RARELY" : 2,
            "NEVER" : 1,
            np.nan : np.nan
        }
        if maxScale == 7:
            return seven_scale_mappings
        return five_scale_mappings

    # if the raw data has "Agree"/"Disagree" in quantitative question columns, map these to integers 1-X where X is "out_of"
    def mapData(self, data):
        for question in self.getQuantQuestions():
            # get the max scale of current question
            maxScale = int(self.question_dict[self.question_dict["question_id"] == question]["out_of"])
            scaleFunction = self.mappingScale(maxScale)
            question_str = str(question)
            
            if maxScale == 7 or maxScale == 5:
                data[question_str] = data[question_str].str.upper()
    
                # starts mapping relevant questions
                lstOfResponses = []
                for response in data[question_str]:
                    lstOfResponses.append(scaleFunction[response])
                data[question_str] = lstOfResponses
        return data
        
    # if normalization is true, then all quantitative data is normed to 0
    # assumed question_dictionary includes "out_of" column
    def normalize(self, data):
        for question in self.getQuantQuestions():
            # get the max scale of current question
            maxScale = int(self.question_dict[self.question_dict["question_id"] == question]["out_of"])
            scaleFunction = self.mappingScale(maxScale)
            question_str = str(question)
            
            # if series values are agree/disagree, adjust the scale for visualization purpose
            if maxScale == 7:
                data[question_str] = pd.to_numeric(data[question_str]) - 4
            else:
                data[question_str] = pd.to_numeric(data[question_str]) - 3
        return data

    # For NA values (students that didn't complete survey, left question empty), fill in with "No Response"
    def fillNA(self, data):
        data = data.fillna('No Response')
        return data

    def main(self):
        self.assertCSVFiles()
        # reads CSVs
        self.readCSVs()

        # joins subset of roster and raw data
        data = self.joinRosterAndRaw()
        data = self.stringToNumerical(data)
        
        if self.needsMapping:
            data = self.mapData(data)

        if self.needsNormalization:
            data = self.normalize(data)
        data = self.sortColumns(data)
        return data

In [162]:
# stores the result that the previous cleaning function generates
oldFileResult = cleanData('TEST_E29_RAW_NEEDSMAPPING.csv', 'TEST_E29_QUESTION_DICTIONARY.csv', 'TEST_E29_ROSTER.csv')
oldFileDF = oldFileResult.main()
oldFileDF

/var/folders/wn/p245w4_d07z716yzmy4c4nlc0000gn/T/ipykernel_1667/2433437924.py:127: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  maxScale = int(self.question_dict[self.question_dict["question_id"] == question]["out_of"])
/var/folders/wn/p245w4_d07z716yzmy4c4nlc0000gn/T/ipykernel_1667/2433437924.py:146: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  maxScale = int(self.question_dict[self.question_dict["question_id"] == question]["out_of"])


,Email,TeamNumber,TeammateNumber,RecipientLastName,RecipientFirstName,2.1,2.2,2.3,3.0,4.1,...,19.3,19.4,19.5,19.6,33.0,34.0,38.1,38.2,38.3,41.0
0,feynmanbarney@berkeley.edu,101-1,1.0,Barney,Feynman,2.0,3.0,3.0,Create a project that follows all of the guide...,3.0,...,2.0,2.0,2.0,2.0,Yes,I was more considerate of my teammates needs a...,1.0,1.0,1.0,Seems good to me
1,smitbhagat@berkeley.edu,101-1,2.0,Bhagat,Smit,3.0,3.0,3.0,NaN,3.0,...,3.0,3.0,3.0,3.0,Yes,NaN,3.0,3.0,3.0,NaN
2,shivdhillon@berkeley.edu,101-1,3.0,Dhillon,Shivreet,3.0,3.0,3.0,NaN,3.0,...,3.0,3.0,3.0,3.0,Yes,NaN,3.0,3.0,3.0,NaN
3,pratik.g@berkeley.edu,101-1,4.0,Gandhi,Pratik,3.0,3.0,3.0,be quicker in meeting and plan beforehand,2.0,...,3.0,3.0,3.0,-2.0,Yes,improved on them,3.0,3.0,3.0,no
4,dawood.junaid@berkeley.edu,101-1,5.0,Junaid,Dawood,3.0,2.0,2.0,"At this point in time, the entire team wants t...",1.0,...,3.0,3.0,3.0,2.0,Yes,None!,2.0,2.0,2.0,None!
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,roshniarun@berkeley.edu,104-6,1.0,Arun,Roshni,3.0,3.0,3.0,Collaboration to finish our gosl,2.0,...,2.0,2.0,2.0,NaN,Yes,NaN,2.0,2.0,2.0,NaN
127,kiliancox@berkeley.edu,104-6,2.0,Cox,Kilian,2.0,2.0,2.0,"Create an effective prototype, and develop a d...",2.0,...,2.0,2.0,2.0,2.0,Yes,NaN,1.0,2.0,1.0,NaN
128,dylan.jemtegaard@berkeley.edu,104-6,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,jasonkhat@berkeley.edu,104-6,4.0,Khatkar,Jason,2.0,2.0,NaN,To do well in our final project together,1.0,...,0.0,3.0,3.0,NaN,Yes,We made a much more solid plan for the last le...,1.0,2.0,-1.0,Maybe don't require me to download software an...
